In [1]:
import numpy as np
import gym
import random
import keras

from collections import deque
from keras import Sequential
from keras.layers import Dense, InputLayer
from keras.models import Model
from time import sleep

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Попробуем собрать DQN для простой среды - перевернутого маятника

In [2]:
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


Вот так она выглядит:

![img](https://i.ytimg.com/vi/5SEEwqRH8_c/hqdefault.jpg)

In [3]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

Вот так выглядят случайные действия:

In [4]:
env = gym.make('CartPole-v0')
env.reset()
env.render()
sleep(0.1)
done = False
while done != True:
    a = env.action_space.sample()
    s2, r, done, info = env.step(a)
    env.render()
    sleep(0.1)
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
# !pip install pyglet==1.2.4

Опишем агента, которого мы будем обучать:

In [6]:
class Agent:
    def __init__(self, env, hidden_dims=[32, 32], lr=0.01):
        '''
        Для инициализации агента передаем среду из gym,
        размеры скрытых слоев сети hidden_dims, 
        и скорость обучения lr
        '''
        self.env = env
        self.input_dim = env.observation_space.shape[0]
        self.output_dim = env.action_space.n
        self.create_model(hidden_dims, lr)

    def create_model(self, hidden_dims, lr):
        '''
        Описываем полносвязную нейронную сеть, у которой количесвто входов - 
        размерность состояния, количество выходов - количество действий.
        Количесвтво внутренних скрытых слоев передаем в hidden_dims.
        '''
        self.model = Sequential()
        self.model.add(InputLayer(input_shape=(self.input_dim,)))
        for hidden_dim in hidden_dims:
            self.model.add(Dense(units=hidden_dim, activation='relu'))
        self.model.add(Dense(units=self.output_dim))
        self.model.compile(loss=keras.losses.mse,
                           optimizer=keras.optimizers.SGD(lr))

    def act(self, X, eps=1.0):
        '''
        Выбираем действие для состояния X.
        Действуем эпсилон-жадно с заданным eps.
        '''
        if np.random.rand() < eps:
            return self.env.action_space.sample()
        X = X.reshape(1, self.input_dim)
        Q = self.model.predict_on_batch(X)
        return np.argmax(Q, 1)[0]

    def train(self, X_batch, y_batch):
        '''
        Делаем шаг обучения для батча состояний X_batch и таргетов y_batch.
        '''
        return self.model.train_on_batch(X_batch, y_batch)

    def predict(self, X_batch):
        '''
        Делаем предсказания Q-значений для батча состояний X_batch.
        '''
        return self.model.predict_on_batch(X_batch)

In [7]:
def create_batch(agent, memory, batch_size, gamma):
    '''
    Функция собирает батч размера batch_size из памяти memory
    для агента agent c коэффициентом дисконтирования gamma.
    Для сделанного действия таргет считаем по алгоритму Q-обучения, 
    для остальных берем предсказание агента.
    Функция возвращает батч состояний X_batch и батч таргетов y_batch.
    '''
    sample = random.sample(memory, batch_size)
    sample = np.asarray(sample)

    s = sample[:, 0]
    a = sample[:, 1].astype(np.int8)
    r = sample[:, 2]
    s2 = sample[:, 3]
    d = sample[:, 4] * 1.

    X_batch = np.vstack(s)
    y_batch = agent.predict(X_batch)

    y_batch[np.arange(batch_size), a] = r + gamma * np.max(agent.predict(np.vstack(s2)), 1) * (1 - d)

    return X_batch, y_batch

In [8]:
def show_agent_play(env_name, agent, eps):
    '''
    Визуализация поведения в gym среде с названием env_name 
    агента agent по эпсилон-жадной стратегии с заданным eps.
    Функция возвращает сумарный reward показанного эпизода.
    '''
    env = gym.make(env_name)
    s = env.reset()
    env.render()
    done = False
    episode_reward = 0
    while done != True:
        a = agent.act(s, eps)
        s2, r, done, info = env.step(a)
        episode_reward += r
        env.render()
        s = s2
        sleep(0.01)
    env.close()
    return episode_reward

In [9]:
env = gym.make('CartPole-v0')
agent = Agent(env)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [10]:
show_agent_play('CartPole-v0', agent, 0)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


9.0

Обучим описанного агента для выбранной среды:

In [12]:
n_episode = 1000
gamma = 0.9
n_memory = 50000
batch_size = 100
eps = 0.5
eps_decay = 0.98
min_eps = 0.05
env_name = 'CartPole-v0'
env = gym.make(env_name)
agent = Agent(env)
memory = deque()
# Условием окончания игры будет определенное среденее значение награды за 30 эпизодов
LAST_30_GAME_EPISODE_REWARDS = deque()
target_reward = 195.0

for episode in range(n_episode):
    done = False
    s = env.reset()
    episode_reward = 0
    while not done:
        a = agent.act(s, eps)
        s2, r, done, info = env.step(a)
        episode_reward += r

        memory.append([s, a, r, s2, done])

        if len(memory) > n_memory:
            memory.popleft()

        if len(memory) > batch_size:
            X_batch, y_batch = create_batch(agent, memory, batch_size, gamma)
            agent.train(X_batch, y_batch)

        s = s2

    print('[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}')
    
    LAST_30_GAME_EPISODE_REWARDS.append(episode_reward)
    if len(LAST_30_GAME_EPISODE_REWARDS) > 30:
        LAST_30_GAME_EPISODE_REWARDS.popleft()

    if np.mean(LAST_30_GAME_EPISODE_REWARDS) >= target_reward:
        print("Game solved in {episode + 1} steps with average reward {np.mean(LAST_30_GAME_EPISODE_REWARDS)}")
        break
    
    eps = max(min_eps, eps*eps_decay)

env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode:>5}] Reward: {episode_reward:>5} EPS: {eps:>3.2f}
[Episode {episode

Посмотрим как действует обученный агент:

In [13]:
show_agent_play(env_name, agent, eps=0.)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


200.0

In [14]:
show_agent_play(env_name, agent, eps=0.3)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


200.0

In [15]:
show_agent_play('CartPole-v1', agent, eps=0.)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


247.0

Видим, что как только среда расширяется - наш агент не может адекватно с ней дружить.

Что тут еще можно попробовать, что должно работать на любой системе:
* обучиться в CartPole-v1 до скора 495
* попробовать поиграть в [Acrobot-v1](https://gym.openai.com/envs/Acrobot-v1/) до скора -120

Если хочется нырнуть поглубже, то можно попрбовать посадить [LunarLander-v2](https://gym.openai.com/envs/LunarLander-v2/) или поиграть в [Atari](https://gym.openai.com/envs/#atari) - там для каждой из игр есть версия где нужно учиться по картинке и версия где нужно учиться по реестровым численным данным приставки. Но для этих сред понадобятся библиотеки box2d-py и atari-py, с которыми все будет хорошо под unixовыми системами, а вот под виндой придется плясать с бубном.